In [1]:
import pandas as pd
import pyodbc
import mysql.connector

conn = mysql.connector.connect(
    host="127.0.0.1",       
    user="root",            
    password="sachin@0210", 
    database="gravity_books"   
)

# query
query = """
SELECT 
    a.book_id, 
    a.title, 
    a.isbn13, 
    a.publication_date, 
    b.language_name,    
    c.publisher_name,   
    d.customer_id,
    e.order_id,    
	h.author_name
FROM 
    book a
JOIN 
    book_language b ON a.language_id = b.language_id
JOIN 
    publisher c  ON a.publisher_id = c.publisher_id
JOIN 
    order_line f ON a.book_id = f.book_id        
JOIN 
    cust_order e ON e.order_id = f.order_id      
JOIN 
    customer d ON d.customer_id = e.customer_id
JOIN
	book_author g ON a.book_id = g.book_id
JOIN 
	author h on g.author_id = h.author_id order by a.book_id asc;
"""

# Execute 
result = pd.read_sql_query(query, conn)
books_df = pd.DataFrame(result)
# Close connection
conn.close()

books_df


C:\Users\c186\AppData\Local\Temp\ipykernel_9220\3694041529.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, conn)


,book_id,title,isbn13,publication_date,language_name,publisher_name,customer_id,order_id,author_name
0,3,The Servant Leader,23755004321,2003-03-11,English,Thomas Nelson,1023,6428,Kenneth H. Blanchard
1,3,The Servant Leader,23755004321,2003-03-11,English,Thomas Nelson,312,1294,Kenneth H. Blanchard
2,4,What Life Was Like in the Jewel in the Crown: ...,34406054602,1999-09-01,English,Time Life Medical,1508,4555,Time-Life Books
3,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,1983-12-29,English,Cliffs Notes,1710,7018,W. John Campbell
4,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,1983-12-29,English,Cliffs Notes,1088,2230,W. John Campbell
...,...,...,...,...,...,...,...,...,...
24526,11123,Asfixia,9789875661707,2006-09-01,English,Debolsillo,220,1090,Chuck Palahniuk
24527,11123,Asfixia,9789875661707,2006-09-01,English,Debolsillo,1017,6642,Chuck Palahniuk
24528,11125,Fantasmas,9789879397503,2006-08-28,Spanish,Mondadori,75,11420,Chuck Palahniuk
24529,11125,Fantasmas,9789879397503,2006-08-28,Spanish,Mondadori,75,11420,Javier Calvo


In [199]:
books_df.columns

Index(['book_id', 'title', 'isbn13', 'publication_date', 'language_name',
       'publisher_name', 'customer_id', 'order_id', 'author_name'],
      dtype='object')

In [4]:
books_df.reset_index(inplace=True)

In [2]:
# convert date time formate
books_df["publication_date"] = pd.to_datetime(books_df["publication_date"])
books_df["published_year"] = books_df["publication_date"].dt.year

In [3]:
# drop publication_date Column
books_df.drop("publication_date",axis=1,inplace=True)

In [147]:
books_df.head()

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year
0,3,The Servant Leader,23755004321,English,Thomas Nelson,1023,6428,Kenneth H. Blanchard,2003
1,3,The Servant Leader,23755004321,English,Thomas Nelson,312,1294,Kenneth H. Blanchard,2003
2,4,What Life Was Like in the Jewel in the Crown: ...,34406054602,English,Time Life Medical,1508,4555,Time-Life Books,1999
3,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,English,Cliffs Notes,1710,7018,W. John Campbell,1983
4,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,English,Cliffs Notes,1088,2230,W. John Campbell,1983


In [88]:
books_df.isnull().sum()

book_id           0
title             0
isbn13            0
language_name     0
publisher_name    0
customer_id       0
order_id          0
author_name       0
published_year    0
dtype: int64

In [30]:
books_df.columns

Index(['book_id', 'title', 'isbn13', 'language_name', 'publisher_name',
       'customer_id', 'order_id', 'author_name', 'published_year'],
      dtype='object')

# **Genres**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Combine metadata
books_df['combined_metadata'] = books_df['title'] + " " + books_df['author_name'] + " " + books_df['publisher_name']

# Vectorize text
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(books_df['combined_metadata'])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=10, random_state=42)  
books_df['genre_cluster'] = kmeans.fit_predict(X)

# Inspect clusters to infer genres
for cluster_id in range(10):
    print(f"Cluster {cluster_id}:")
    print(books_df[books_df['genre_cluster'] == cluster_id]['title'].head(10))


In [5]:
def infer_genre(metadata):
    metadata = metadata.lower()
    
    # Define keywords for each genre
    genre_keywords = {
        "Romance": ["romance", "love", "affair", "relationship", "valentine", "passion"],
        "Science Fiction": ["sci-fi", "science fiction", "space", "alien", "robot", "futuristic", "time travel"],
        "Fantasy": ["fantasy", "magic", "wizard", "dragon", "myth", "sorcery", "elves"],
        "Mystery/Thriller": ["mystery", "thriller", "detective", "crime", "investigation", "murder", "suspense"],
        "Historical": ["history", "historical", "past", "war", "empire", "ancient", "renaissance"],
        "Horror": ["horror", "ghost", "supernatural", "haunted", "fear", "zombie", "vampire", "monster"],
        "Biography": ["biography", "memoir", "life story", "autobiography", "personal", "inspiration"],
        "Self-Help": ["self-help", "motivation", "personal development", "improvement", "success", "habit"],
        "Children's Literature": ["children", "kids", "juvenile", "fairy tale", "nursery", "storybook"],
        "Adventure": ["adventure", "exploration", "journey", "expedition", "quest", "survival"],
        "Non-Fiction": ["non-fiction", "true story", "real life", "documentary", "facts"],
        "Fiction": ["fiction", "novel", "story", "literature"],
        "Poetry": ["poetry", "poem", "verse", "lyric", "haiku"],
        "Young Adult": ["young adult", "teen", "high school", "coming of age"],
        "Spiritual/Religion": ["spiritual", "religion", "faith", "philosophy", "bible", "belief"],
        "Comedy/Humor": ["comedy", "humor", "funny", "satire", "parody", "joke"],
        "Business/Economics": ["business", "economics", "management", "finance", "entrepreneurship", "leadership"],
        "Technology": ["technology", "programming", "software", "ai", "artificial intelligence", "computing"],
        "Education/Academic": ["education", "academic", "study", "research", "textbook", "learning"],
        "Health/Fitness": ["health", "fitness", "wellness", "diet", "exercise", "nutrition"],
    }
    
    # Search for keywords in metadata
    for genre, keywords in genre_keywords.items():
        for keyword in keywords:
            if keyword in metadata:
                return genre
    
    # Default genre if no match is found
    return "General"

# Apply the function to the combined metadata column
books_df['genres'] = books_df['combined_metadata'].apply(infer_genre)


In [175]:
books_df

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year,combined_metadata,genre_cluster,genres
0,3,The Servant Leader,23755004321,English,Thomas Nelson,1023,6428,Kenneth H. Blanchard,2003,The Servant Leader Kenneth H. Blanchard Thomas...,1,General
1,3,The Servant Leader,23755004321,English,Thomas Nelson,312,1294,Kenneth H. Blanchard,2003,The Servant Leader Kenneth H. Blanchard Thomas...,1,General
2,4,What Life Was Like in the Jewel in the Crown: ...,34406054602,English,Time Life Medical,1508,4555,Time-Life Books,1999,What Life Was Like in the Jewel in the Crown: ...,9,General
3,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,English,Cliffs Notes,1710,7018,W. John Campbell,1983,Cliffs Notes on Aristophanes' Lysistrata The ...,1,General
4,5,Cliffs Notes on Aristophanes' Lysistrata The ...,49086007763,English,Cliffs Notes,1088,2230,W. John Campbell,1983,Cliffs Notes on Aristophanes' Lysistrata The ...,1,General
...,...,...,...,...,...,...,...,...,...,...,...,...
24526,11123,Asfixia,9789875661707,English,Debolsillo,220,1090,Chuck Palahniuk,2006,Asfixia Chuck Palahniuk Debolsillo,1,General
24527,11123,Asfixia,9789875661707,English,Debolsillo,1017,6642,Chuck Palahniuk,2006,Asfixia Chuck Palahniuk Debolsillo,1,General
24528,11125,Fantasmas,9789879397503,Spanish,Mondadori,75,11420,Chuck Palahniuk,2006,Fantasmas Chuck Palahniuk Mondadori,1,General
24529,11125,Fantasmas,9789879397503,Spanish,Mondadori,75,11420,Javier Calvo,2006,Fantasmas Javier Calvo Mondadori,1,General


In [6]:
# drop publication_date Column
books_df.drop(["combined_metadata","genre_cluster"],axis=1,inplace=True)

In [ ]:
# Save data farme

#books_df.to_csv("Final_data.csv",index=False)

In [136]:
books_df.columns

Index(['book_id', 'title', 'isbn13', 'language_name', 'publisher_name',
       'customer_id', 'order_id', 'author_name', 'published_year', 'genres'],
      dtype='object')

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

books_df['customer_id'] = encoder.fit_transform(books_df['customer_id'])
books_df['title'] = encoder.fit_transform(books_df['title'])
books_df['language_name'] = encoder.fit_transform(books_df['language_name'])
books_df['publisher_name'] = encoder.fit_transform(books_df['publisher_name'])
books_df['author_name'] = encoder.fit_transform(books_df['author_name'])
books_df['published_year'] = encoder.fit_transform(books_df['published_year'])
books_df['genres'] = encoder.fit_transform(books_df['genres'])


In [8]:
x=books_df.drop("genres",axis=1)
#y=books_df["genres"]

In [9]:
# One-hot encoding for target labels (if target is multi-class classification)
num_classes = len(books_df['genres'].unique())
y = pd.get_dummies(books_df['genres']).values

In [207]:
# from  sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(books_df.drop("genres",axis=1),books_df["genres"],test_size=0.2,random_state=1)

In [208]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((19624, 9), (4907, 9), (19624,), (4907,))

In [209]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)
# x_test = sc.transform(x_test)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [18]:
# Model architecture
model = Sequential()

# Input layer: The number of features in `X`
model.add(Dense(128, input_dim=X.shape[1], activation="relu"))  # Input layer

# Hidden layers
model.add(Dense(64, activation="relu"))  # HL1
model.add(Dense(64, activation="relu"))  # HL2
model.add(Dense(32, activation="relu"))  # HL3
model.add(Dense(16, activation="relu"))  # HL4

# Output layer: Number of classes for prediction
model.add(Dense(num_classes, activation="softmax"))  # Output layer

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=[
        tf.keras.metrics.F1Score(),  #(num_classes=num_classes, average="macro"),
        "accuracy",
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │       640,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 21)             │           357 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 655,509 (2.50 MB)

 Trainable params: 655,509 (2.50 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Train the model
model.fit(X_train,y_train,epochs=50,batch_size=32,validation_data=(X_test, y_test))

Epoch 1/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6696 - f1_score: 0.0606 - loss: 1.5683 - precision: 0.7656 - recall: 0.4996 - val_accuracy: 0.8272 - val_f1_score: 0.2863 - val_loss: 0.6288 - val_precision: 0.9405 - val_recall: 0.7532
Epoch 2/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8863 - f1_score: 0.4598 - loss: 0.3942 - precision: 0.9553 - recall: 0.8481 - val_accuracy: 0.9305 - val_f1_score: 0.7196 - val_loss: 0.2890 - val_precision: 0.9528 - val_recall: 0.9134
Epoch 3/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9742 - f1_score: 0.8088 - loss: 0.1098 - precision: 0.9833 - recall: 0.9662 - val_accuracy: 0.9440 - val_f1_score: 0.8050 - val_loss: 0.2684 - val_precision: 0.9539 - val_recall: 0.9407
Epoch 4/50
614/614 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9856 - f1_score: 0.8928 - loss: 0.0588 - precision: 0.9901 - recall: 0.9843 - val_accuracy: 0.9527 - val_f1_score: 0.8546 - val_loss: 0.2636 - val_precision: 0.9583 - val_recall:

In [21]:
# Evaluate the model
loss, f1_score, accuracy, precision, recall = model.evaluate(X_test, y_test)

print(f"Loss: {loss}")
print(f"F1 Score: {f1_score}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9562 - f1_score: 0.8671 - loss: 0.5426 - precision: 0.9583 - recall: 0.9556
Loss: 0.518930971622467
F1 Score: [0.8727272  0.87999994 0.91999996 0.972477   0.96       0.8771929
 0.92096215 0.91329473 0.975911   0.83333325 0.93374425 0.9291338
 0.95808387 0.5555555  0.9503545  0.92028975 0.95808375 0.74999994
 0.9642856  0.8875912  0.8979591 ]
Accuracy: 0.9563888311386108
Precision: 0.9585206508636475
Recall: 0.9559812545776367


In [23]:
import numpy as np

# Predict
predictions = model.predict(X_test)

# Decode the prediction back to labels
predicted_genres = np.argmax(predictions, axis=1)
print("Predicted genres:", predicted_genres)


154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Predicted genres: [19  8  6 ...  8 10 19]


In [24]:
import pickle

# Save the model to a pickle file
model_path = "book_recommendation_model.pkl"
with open(model_path, "wb") as file:
    pickle.dump(model, file)

In [179]:
from keras.utils import to_categorical 

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y) 

# Apply one-hot encoding
y_new = to_categorical(y_encoded)

In [180]:
books_df

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year,genres
0,3,6394,23755004321,6,1582,852,6428,3890,63,8
1,3,6394,23755004321,6,1582,261,1294,3890,63,8
2,4,7235,34406054602,6,1588,1275,4555,6884,59,8
3,5,1091,49086007763,6,330,1445,7018,7101,43,8
4,5,1091,49086007763,6,330,907,2230,7101,43,8
...,...,...,...,...,...,...,...,...,...,...
24526,11123,534,9789875661707,6,408,185,1090,1109,66,8
24527,11123,534,9789875661707,6,408,846,6642,1109,66,8
24528,11125,1823,9789879397503,20,1021,58,11420,1109,66,8
24529,11125,1823,9789879397503,20,1021,58,11420,3076,66,8


In [181]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y_new,test_size=0.25)

In [157]:
x_train

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year
3491,1566,6447,9780143037491,6,1170,901,2321,3112,66
522,290,2463,9780060581060,6,121,13,4642,6664,65
24133,10943,1085,9788408065067,20,1234,995,9454,6919,66
12790,6198,2382,9780679800033,6,1320,1619,6671,1722,50
12841,6217,1594,9780679890706,6,1334,1678,9007,6214,70
...,...,...,...,...,...,...,...,...,...
3280,1489,1888,9780142405499,6,548,344,10369,3756,66
159,99,4069,9780007178483,6,572,1377,5085,492,64
4875,2135,3678,9780307264220,23,860,624,8367,2542,66
3154,1441,3672,9780142003251,6,1176,147,4220,2718,63


In [182]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler()

In [183]:
x_train_scaler = scaler.transform(x_train)
x_test_scaler = scaler.transform(x_test)

In [184]:
x_train_scaler

array([[ 0.70588477,  1.00265726,  0.05088731, ...,  0.88651852,
        -0.90839289,  0.72011428],
       [-1.56590186,  0.82636106,  0.04932718, ...,  1.00024228,
        -1.50003052, -0.38548672],
       [ 1.01591034, -1.35108454,  0.05213946, ..., -0.08899136,
         0.4720949 ,  0.84295884],
       ...,
       [ 0.80014723,  0.76540759,  0.05101019, ..., -0.08641972,
         1.27572446, -0.01695306],
       [ 0.65385678,  0.18775621,  0.05088313, ...,  0.8376573 ,
         0.40235463,  0.59726972],
       [ 0.8136133 , -1.20854719,  0.05102233, ...,  1.59257737,
         1.63689591, -1.49108773]])

In [191]:
x_train_scaler_df=pd.DataFrame(x_train_scaler,columns = x_train.columns)
x_test_scaler_df=pd.DataFrame(x_test_scaler,columns = x_test.columns)


In [185]:
x_train_scaler_df

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year
0,-1.272985,1.238241,0.046122,-0.393420,0.404887,0.154391,-0.874004,-0.278479,0.717801
1,-1.665189,-0.624224,0.045940,-0.393420,-1.537045,-1.648831,-0.210219,1.370143,0.594682
2,1.609219,-1.268419,0.064437,2.021017,0.523365,0.345273,1.165968,1.488498,0.717801
3,0.150751,-0.662090,0.047312,-0.393420,0.682570,1.612402,0.370056,-0.923631,-1.252101
4,0.156591,-1.030469,0.047312,-0.393420,0.708488,1.732210,1.038131,1.161280,1.210276
...,...,...,...,...,...,...,...,...,...
18393,-1.296653,-0.893028,0.046121,-0.393420,-0.746573,-0.976684,1.427650,0.020427,0.717801
18394,-1.723896,0.126559,0.045821,-0.393420,-0.702144,1.120983,-0.083525,-1.494522,0.471563
18395,-1.098092,-0.056228,0.046486,2.538396,-0.168992,-0.408100,0.855096,-0.543038,0.717801
18396,-1.311406,-0.059033,0.046120,-0.393420,0.415994,-1.376723,-0.330907,-0.461349,0.348444


In [163]:
x_test_scaler_df

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year
0,0.490701,-1.360047,0.047507,-0.393420,-1.283427,0.938224,-1.270101,-1.432792,0.471563
1,1.047348,0.799739,0.048954,2.538396,-0.942802,0.048797,1.197141,-1.492666,0.594682
2,0.060691,-0.228730,0.047294,-1.083259,0.551134,-0.036490,-0.094965,1.053600,-1.128982
3,-0.601383,1.488813,0.046682,-0.393420,-0.307833,-0.921856,1.167970,0.942671,-1.375220
4,0.049319,-0.757925,0.047293,-0.393420,0.551134,0.944316,-1.398797,-0.903673,-0.513388
...,...,...,...,...,...,...,...,...,...
6128,-1.563757,-1.331062,0.045941,-0.393420,-0.470741,0.599105,1.447669,1.428624,-0.144031
6129,0.380663,1.263485,0.047462,-0.393420,0.269748,1.701750,0.339455,-0.278014,0.348444
6130,1.123268,0.565996,0.049271,-0.393420,1.363820,0.412284,0.525063,-0.173583,-0.882744
6131,1.400208,-0.055293,0.049891,-0.393420,-0.359668,-0.412162,-0.922908,-1.289837,0.225325


In [186]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

model = Sequential()

# Hidden layers
model.add(Dense(128,input_dim = x_train_scaler_df.shape[1],activation='relu'))
model.add(Dense(64, activation='relu'))   # Hidden layer 2
model.add(Dense(32, activation='relu'))   # Hidden layer 3
model.add(Dense(16, activation='relu'))   # Hidden layer 4

# Output layer
model.add(Dense(3, activation='softmax'))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=[tf.keras.metrics.F1Score(),"accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

c:\Users\c186\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,195 (47.64 KB)

 Trainable params: 12,195 (47.64 KB)

 Non-trainable params: 0 (0.00 B)

In [196]:
import numpy as np

# Ensure features are float32
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [197]:
#result = model.fit(x_train_scaled_df,y_train,epochs=50)

result = model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 21), output.shape=(None, 3)

In [189]:
x

,book_id,title,isbn13,language_name,publisher_name,customer_id,order_id,author_name,published_year
0,3,6394,23755004321,6,1582,852,6428,3890,63
1,3,6394,23755004321,6,1582,261,1294,3890,63
2,4,7235,34406054602,6,1588,1275,4555,6884,59
3,5,1091,49086007763,6,330,1445,7018,7101,43
4,5,1091,49086007763,6,330,907,2230,7101,43
...,...,...,...,...,...,...,...,...,...
24526,11123,534,9789875661707,6,408,185,1090,1109,66
24527,11123,534,9789875661707,6,408,846,6642,1109,66
24528,11125,1823,9789879397503,20,1021,58,11420,1109,66
24529,11125,1823,9789879397503,20,1021,58,11420,3076,66


In [190]:
y

0        8
1        8
2        8
3        8
4        8
        ..
24526    8
24527    8
24528    8
24529    8
24530    8
Name: genres, Length: 24531, dtype: int32

In [ ]:
model=Sequential()
#input dimension is 4 hence 4 neurons - developer scope
model.add(Dense(256,input_dim=12,activation='elu')) # hidden layer 1
model.add(Dense(128,activation='swish')) # hidden layer 2
model.add(Dense(64,activation='tanh'))# hidden layer 3
model.add(Dense(32,activation='tanh'))# hidden layer 4
model.add(Dense(16,activation='tanh'))# hidden layer 5
model.add(Dense(7,activation='softmax'))# output layer
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=(['accuracy'],['confusion_matrix'],['Precision'],['Recall'],['f1_score']))

c:\Users\c186\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# training
model.fit(x_train,y_train,epochs=10)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
import pandas as pd

# Example preprocessing
books_df['year'] = pd.to_datetime(books_df['publication_date']).dt.year
books_df['language_code'] = books_df['language_name'].factorize()[0]

user_books = books_df.groupby('customer_id')['book_id'].apply(list).reset_index()



In [15]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

books_df['user_encoded'] = user_encoder.fit_transform(books_df['customer_id'])
books_df['book_encoded'] = book_encoder.fit_transform(books_df['book_id'])

interactions = books_df[['user_encoded', 'book_encoded']]


In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(interactions, test_size=0.2, random_state=42)


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

num_users = books_df['user_encoded'].nunique()
num_books = books_df['book_encoded'].nunique()
embedding_dim = 50

# User and book input layers
user_input = Input(shape=(1,))
book_input = Input(shape=(1,))

# Embedding layers
user_embedding = Embedding(num_users, embedding_dim)(user_input)
book_embedding = Embedding(num_books, embedding_dim)(book_input)

# Flatten embeddings
user_vec = Flatten()(user_embedding)
book_vec = Flatten()(book_embedding)

# Dot product for similarity
similarity = Dot(axes=1)([user_vec, book_vec])

# Output layer
output = Dense(1, activation='sigmoid')(similarity)

# Model
model = Model(inputs=[user_input, book_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF for titles
tfidf = TfidfVectorizer(max_features=5000)
book_features = tfidf.fit_transform(books_df['title'])

# Example deep learning model
book_input = tf.keras.Input(shape=(book_features.shape[1],))
dense = tf.keras.layers.Dense(128, activation='relu')(book_input)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.Model(inputs=book_input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
# Ensure 'interacted' column exists in the dataset
train_labels = train['interacted'].values
test_labels = test['interacted'].values


KeyError: 'interacted'

In [19]:
# Example training
history = model.fit(
    [train['user_encoded'], train['book_encoded']],
    train_labels,
    batch_size=64,
    epochs=10,
    validation_data=([test['user_encoded'], test['book_encoded']], test_labels)
)


NameError: name 'train_labels' is not defined